In [ ]:
using InstantiateFromURL
# optionally add arguments to force installation: instantiate = true, precompile = true
github_project("QuantEcon/quantecon-notebooks-julia", version = "0.8.0")

In [ ]:
using LinearAlgebra, Statistics
using Parameters, QuantEcon, DataFrames, Plots, Random

In [ ]:
"""
Baseline calibration
"""

function Model(;βm = .966,     # Money market interest rate equal to 3.5%
                βf = .939,     # Debt to GDP = 44.4% (BY SIMULATION)
                 σ = 2.0,      # Standard parameter value as Alfaro and Kanczuk (2009)
                 r = 0.011,    # U.S. Treasury Bills real interest rate
                d0 = -0.809,   # Mean spread = 267 bp (BY SIMULATION)
                d1 = 0.902,    # Increase in spread = 296 bp (BY SIMULATION)
                 ρ = 0.66,     # Auto-correlation of Mexico's GDP
                 η = 0.034,    # Variance of Mexico's GDP
                 κ = 0.27,     # Probability of default equal to 2% (BY SIMULATION)
                lh = .3,       # Probability of transiting to high risk premium (Global EMBI+)
                hl = .80,      # Probability of transiting to low risk premium (Global EMBI+)
                 θ = 0.11,     # 9 years in default for Mexico (1982-1990)
                 g = 0.,       # Standard parameter value as Alfaro and Kanczuk (2009)
                ny = 11,       
                nB = 31,
                nA = 11)

    # create grids
    Bgrid = collect(range(0, .6, length = nB))
    Agrid = collect(range(0, .2, length = nA))
    mc = tauchen(ny, ρ, η)
    Πy = mc.p
    ygrid = exp.(mc.state_values)
    
    # Income loss specification as in Chaterjee and Eyingungor
     ydefcost = zeros(ny)
     for iy=1:ny
         ydefcost[iy] = max(0, d0*ygrid[iy] + d1*(ygrid[iy]^2))
     end
     ydefgrid = ygrid .- ydefcost
    
    # Utility loss specification as in Bianchi, Hatchondo, and Martinez (2018)
    #udefgrid = zeros(ny)
    #for iy=1:ny
    #    udefgrid[iy] = d0 + d1*log(ygrid[iy])
    #end
    
    # Transition matrix for the risk premium shocks
    Πκ = [1-lh lh; hl 1-hl]
    
    # define value functions
    # notice ordered different than Python to take
    # advantage of column major layout of Julia)
    vmd = zeros(1, ny, nA, 2)
    vmr = zeros(nB, ny, nA, 2)
    vf = zeros(nB, ny, nA, 2)
    vfd = zeros(1, ny, nA, 2)
    vfr = zeros(nB, ny, nA, 2)
    policyAd = ones(Int64, 1, ny, nA, 2)
    policyAr = ones(Int64, nB, ny, nA, 2)
    policyB = ones(Int64, nB, ny, nA, 2)
    policyD = zeros(Int64, nB, ny, nA, 2)
    q = ones(nB, ny, nA, 2) .* exp(-r)
    defprob = zeros(nB, ny, nA, 2)

    return (βm = βm, βf = βf, σ = σ, r = r, d0 = d0, d1 = d1, ρ = ρ, η = η, 
            κ = κ, lh = lh, hl = hl, θ = θ, g = g, ny = ny, nB = nB, nA = nA, 
            ygrid = ygrid, ydefgrid=ydefgrid, Bgrid = Bgrid, Agrid = Agrid, Πy = Πy, Πκ = Πκ, 
            vmd = vmd, vmr = vmr, vf = vf, vfd = vfd, vfr = vfr, policyAd = policyAd, policyAr = policyAr, 
            policyB = policyB, policyD = policyD, q = q, defprob = defprob)
end

"""
UTILITY FUNCTION
"""
u(ae, c) = (c^(1 - ae.σ)) / (1 - ae.σ)

"""
THIS FUNCTION UPDATES THE MONETARY AUTHORITY'S VALUE FUNCTION IN DEFAULT
"""
function Md_one_step_update!(ae,Evmd,Evmr)

    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae
    
    # For normal and risky times
    for iκ=1:2                                         
        # For each endowment shock
        for iy=1:ny
            ydef = ydefgrid[iy]
            # For each initial level of reserves
            for ia=1:nA
                # initial level of reserves
                A = Agrid[ia]
                # optimal choice of A' in default states
                current_max = -Inf
                Aprime = 1
                for iaprime=1:nA
                    # consumption of public goods in default
                    cdef = max(ydef - g + A - exp(-r)*Agrid[iaprime], 0.001)  
                    # value of choosing A' equal to iaprime
                    v = u(ae, cdef) +  βm * (θ * Evmr[1,iy,iaprime,iκ] + (1-θ) * Evmd[1,iy,iaprime,iκ])
                    if v > current_max
                        current_max = v
                        Aprime = iaprime
                    end
                end
                # update value and policy functions in default
                vmd[1, iy, ia, iκ] = current_max
                policyAd[1, iy, ia, iκ] = Aprime
            end  
        end
    end
end

"""
THIS FUNCTION SOLVES THE MONETARY AUTHORITY'S RECURSIVE PROBLEM IN DEFAULT USING VALUE FUNCTION ITERATION
"""
function Md_vfi!(ae; tol = 1e-8, maxit = 100)

    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae

    # Iteration stuff
    it = 0
    dist = 10.

    # allocate memory for update
    Vmd_upd = zeros(1, ny, nA, 2)

    while dist > tol && it < maxit
        it += 1

        # compute expectations for this iterations
        Evmd = zeros(1, ny, nA, 2)
        Evmr = zeros(nB, ny, nA, 2)
        
        for ia=1:nA
            for iκ=1:2
                Evmd[1,:,ia,iκ] = ((vmd[1,:,ia,1]' * Πy')*Πκ[iκ,1] + (vmd[1,:,ia,2]' * Πy')*Πκ[iκ,2])'
                Evmr[:,:,ia,iκ] =  (vmr[:,:,ia,1]  * Πy')*Πκ[iκ,1] + (vmr[:,:,ia,2]  * Πy')*Πκ[iκ,2] 
            end          
        end
        
        # Update Monetary authority's value function
        copyto!(Vmd_upd, vmd)
        Md_one_step_update!(ae, Evmd, Evmr)
        dist = maximum(abs(x - y) for (x, y) in zip(Vmd_upd, vmd))
    end
end

"""
THIS FUNCTION UPDATES THE MONETARY AUTHORITY'S VALUE FUNCTION IN REPAYMENT
"""
function Mr_one_step_update!(ae,Evm)

    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae
    
    # For normal and risky times
    for iκ=1:2
        # For each endowment shock
        for iy=1:ny
            y = ygrid[iy]
            # For each initial level of reserves
            for ia=1:nA
                # initial level of reserves
                A = Agrid[ia]
                # For each initial level of debt 
                for ib=1:nB
                    # initial level of debt
                    B = Bgrid[ib]
                    # optimal choice of A' in repayment states
                    current_max = -Inf
                    Aprime = 1
                    for iaprime=1:nA
                        # consumption of public goods in repayment
                        c = max(y - g + A - exp(-r)*Agrid[iaprime] - B
                          + q[policyB[ib,iy,ia,iκ], iy, iaprime, iκ] * Bgrid[policyB[ib,iy,ia,iκ]], 0.001)  
                        # value of choosing A' equal to iaprime
                        v = u(ae, c) + βm * Evm[policyB[ib,iy,ia,iκ], iy, iaprime, iκ]
                        if v > current_max
                            current_max = v
                            Aprime = iaprime
                        end
                    end
                    # update value and policy functions in repayment
                    vmr[ib, iy, ia, iκ] = current_max
                    policyAr[ib, iy, ia, iκ] = Aprime
                end
            end  
        end
    end
end

"""
THIS FUNCTION SOLVES THE MONETARY AUTHORITY'S RECURSIVE PROBLEM IN REPAYMENT USING VALUE FUNCTION ITERATION
"""
function Mr_vfi!(ae; tol = 1e-8, maxit = 100)

    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae

    # Iteration stuff
    it = 0
    dist = 10.

    # allocate memory for update
    Vmr_upd = zeros(nB, ny, nA, 2)

    while dist > tol && it < maxit
        it += 1

        # compute expectations for this iterations
        Evm = zeros(nB, ny, nA, 2)
        
        for ib=1:nB
            for iy=1:ny
                for ia=1:nA
                    for iyprime=1:ny
                        for iκprime=1:2
                            # Normal times today
                            Evm[ib,iy,ia,1] = Evm[ib,iy,ia,1] + ((1-policyD[ib,iyprime,ia,iκprime])*vmr[ib,iyprime,ia,iκprime] 
                            + policyD[ib,iyprime,ia,iκprime]*vmd[1,iyprime,ia,iκprime])*Πy[iy,iyprime]*Πκ[1,iκprime]
                            # Risky times today
                            Evm[ib,iy,ia,2] = Evm[ib,iy,ia,2] + ((1-policyD[ib,iyprime,ia,iκprime])*vmr[ib,iyprime,ia,iκprime] 
                            + policyD[ib,iyprime,ia,iκprime]*vmd[1,iyprime,ia,iκprime])*Πy[iy,iyprime]*Πκ[2,iκprime]
                        end
                    end
                end
            end                      
        end
        
        # Update Monetary authority's value function
        copyto!(Vmr_upd, vmr)
        Mr_one_step_update!(ae, Evm)
        dist = maximum(abs(x - y) for (x, y) in zip(Vmr_upd, vmr))
    end
end

"""
THIS FUNCTION UPDATES THE FISCAL AUTHORITY'S VALUE FUNCTION
"""

function F_one_step_update!(ae, Evfd, Evf)

    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae

    # For normal and risky times
    for iκ=1:2
        # For each endowment shock
        for iy=1:ny
            y    = ygrid[iy]
            ydef = ydefgrid[iy]
            # For each initial level of reserves
            for ia=1:nA
                # initial level of reserves
                A = Agrid[ia]
                # consumption of public goods in default
                cdef = ydef - g + A - exp(-r)*Agrid[policyAd[1,iy,ia,iκ]]
                # value of default
                defval = u(ae, cdef) + βf*(θ*Evf[1,iy,policyAd[1,iy,ia,iκ],iκ] + (1-θ)*Evfd[1,iy,policyAd[1,iy,ia,iκ],iκ])
                # For each initial level of debt 
                for ib=1:nB
                    # initial level of debt
                    B = Bgrid[ib]
                    # optimal choice of B' in repayment states
                    current_max = -Inf
                    Bprime = 1
                    for ibprime=1:nB
                        # consumption of public goods in repayment
                        c = max(y - g + A - exp(-r)*Agrid[policyAr[ib,iy,ia,iκ]] - B
                          + q[ibprime, iy, policyAr[ib,iy,ia,iκ], iκ] * Bgrid[ibprime], 0.001)  
                        # value of choosing B' equal to ibprime
                        v = u(ae, c) + βf * Evf[ibprime, iy, policyAr[ib,iy,ia,iκ], iκ]
                        if v > current_max
                            current_max = v
                            Bprime = ibprime
                        end
                    end
                    # Update value and policy functions in repayment
                    vfd[1, iy, ia, iκ] = defval
                    vfr[ib, iy, ia, iκ] = current_max
                    policyB[ib, iy, ia, iκ] = Bprime

                    # Define default policy function
                    if vfd[1, iy, ia, iκ] > vfr[ib, iy, ia, iκ]
                        vf[ib, iy, ia, iκ] = vfd[1, iy, ia, iκ]
                        policyD[ib, iy, ia, iκ] = 1
                    else
                        vf[ib, iy, ia, iκ] = vfr[ib, iy, ia, iκ]
                        policyD[ib, iy, ia, iκ] = 0
                    end
                end
            end  
        end
    end
end

"""
THIS FUNCTION SOLVES THE FISCAL AUTHORITY'S RECURSIVE PROBLEM USING VALUE FUNCTION ITERATION
"""
function F_vfi!(ae; tol = 1e-8, maxit = 100)

    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae

    # Iteration stuff
    it = 0
    dist = 10.

    # allocate memory for update
    Vf_upd = zeros(nB, ny, nA, 2)

    while dist > tol && it < maxit
        it += 1

        # compute expectations for this iterations
        Evfd = zeros(1, ny, nA, 2)
        Evf  = zeros(nB, ny, nA, 2)
        
        for ia=1:nA
            for iκ=1:2
                Evfd[1,:,ia,iκ] = ((vfd[1,:,ia,1]' * Πy')*Πκ[iκ,1] + (vfd[1,:,ia,2]' * Πy')*Πκ[iκ,2])'
                Evf[:,:,ia,iκ] =  (vf[:,:,ia,1]  * Πy')*Πκ[iκ,1] + (vf[:,:,ia,2]  * Πy')*Πκ[iκ,2] 
            end          
        end  
        
        # Update Monetary authority's value function
        copyto!(Vf_upd, vf)
        F_one_step_update!(ae, Evfd, Evf)
        dist = maximum(abs(x - y) for (x, y) in zip(Vf_upd, vf))
    end
end

"""
THIS FUNCTION UPDATES THE PRICE SCHEDULE
"""
function compute_prices!(ae)
    
    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae
    
    # The probability of tomorrow's default depends on D(B',y',A',κ') given y
    # Notice that prob[ib,iy,ia,iκ] denotes the prob of default tomorrow conditional on (y,κ) and given B' and A'
    proba = zeros(nB, ny, nA, 2)
    qbond = zeros(nB, ny, nA, 2)
    
    for ib=1:nB
        for iy=1:ny
            for ia=1:nA
                for iyprime=1:ny
                    for iκprime=1:2
                        # Normal times today, κ = 0
                        proba[ib,iy,ia,1] = proba[ib,iy,ia,1]
                                          + (policyD[ib,iyprime,ia,iκprime]*Πy[iy,iyprime]*Πκ[1,iκprime])
                        
                        qbond[ib,iy,ia,1] = qbond[ib,iy,ia,1] + ((exp(-r)
                                            *(1-policyD[ib,iyprime,ia,iκprime]))*Πy[iy,iyprime]*Πκ[1,iκprime])
                        
                        # Risky times κ > 0
                        proba[ib,iy,ia,2] = proba[ib,iy,ia,2]
                                          + (policyD[ib,iyprime,ia,iκprime]*Πy[iy,iyprime]*Πκ[2,iκprime])
                        
                        qbond[ib,iy,ia,2] = qbond[ib,iy,ia,2] + ((exp(-r-(.5*(κ^2)*(η^2))- (κ*(ygrid[iyprime]-ρ*ygrid[iy])))
                                            *(1-policyD[ib,iyprime,ia,iκprime]))*Πy[iy,iyprime]*Πκ[2,iκprime])
                    end
                end
            end
        end
    end
    
    # Update probabilities and bond prices
    copyto!(defprob, proba)
    copyto!(q, qbond)
    return

end

"""
This function does the guess and verify for q in the TWO-GOVERNMENT-ENTITIES APPROACH
"""
function Guess_and_Verify!(ae; tol = 1e-8, maxit = 1000)

    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae

    # Iteration stuff for the guess and verify for policyD
    it_q = 0
    dist_q = 10.

    # allocate memory for update
    q0  = ones(nB, ny, nA, 2) .* exp(-r)
    vmr0 = zeros(nB, ny, nA, 2) 

    while dist_q > tol && it_q < maxit
        it_q += 1
        
        # Iteration stuff for the guess and verify for Ar
        it_vmr = 0
        dist_vmr = 10.
        
        while dist_vmr > tol && it_vmr < maxit
            it_vmr +=1
                
            # Solve vmd(y,A) and get Ad(y,A)
            Md_vfi!(ae)
            
            # Solve vf(y,B,A) and get B'(y,B,A) and D(y,B,A)
            F_vfi!(ae)
            
            # Solve vmr(y,B,A) and get Ar(y,B,A)
            Mr_vfi!(ae)
                
            # Distance between guess and reserves policy function
            dist_vmr = maximum(abs(x - y) for (x, y) in zip(vmr0, vmr))
            copyto!(vmr0, vmr)
                
        end
            
        # Compute prices
        compute_prices!(ae)
        
        # Distance between guess and bond prices
        dist_q = maximum(abs(x - y) for (x, y) in zip(q0, q))
        copyto!(q0, q)
        
        if dist_q <= tol
            println("Finished iteration for q $(it_q) with dist of $(dist_q)")
        else
            if it_q%5 == 0
                println("Finished iteration for q $(it_q) with dist of $(dist_q)")
            end
        end
    end
end

In [ ]:
"""
Tables 2-3. 
BENCHMARK CALIBRATION (κ > 0.) PAPER VERSION MAY 2022
INDEPENDENT CENTRAL BANK
βf = 0.939 < βm = .966
"""

samano = Model( βm = .966,     # Money market interest rate equal to 3.5%
                βf = .939,     # Debt to GDP = 44.4% (BY SIMULATION)
                 σ = 2.0,      # Standard parameter value as Alfaro and Kanczuk (2009)
                 r = 0.011,    # U.S. Treasury Bills real interest rate
                d0 = -0.809,   # Mean spread = 267 bp (BY SIMULATION)
                d1 = 0.902,    # Increase in spread = 296 bp (BY SIMULATION)
                 ρ = 0.66,     # Auto-correlation of Mexico's GDP
                 η = 0.034,    # Variance of Mexico's GDP
                 κ = 0.27,     # Probability of default equal to 2% (BY SIMULATION)
                lh = .3,       # Probability of transiting to high risk premium (Global EMBI+)
                hl = .80,      # Probability of transiting to low risk premium (Global EMBI+)
                 θ = 0.11,     # 9 years in default for Mexico (1982-1990)
                 g = 0.,       # Standard parameter value as Alfaro and Kanczuk (2009)
                ny = 11,       
                nB = 31,
                nA = 11)

@time Guess_and_Verify!(samano, tol = 1e-8, maxit = 100)

In [ ]:
"""
This function realizes S simulations of ENDOWMENT and RISK PREMIUM SHOCKS for T periods 
"""
function QuantEcon.simulate(ae, T=1000, S=1000, y0=mean(ae.ygrid))
        
    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae
    
    # Create an array of dim (T,S) to allocate the indices of the endowment shock
    y_indices = zeros(Int64, T+1, S)
    
    # Create an array of dim (T,S) to allocate the indices of the risk premium shock
    κ_indices = zeros(Int64, T+1, S)
    
    # For each simulation s
    for s=1:S
        # Get initial indices
        y0_ind = searchsortedfirst(ygrid, y0)
        κ0_ind = 1
        # Create a QE MarkovChain
        mc_y = MarkovChain(Πy)
        y_indices[:,s] = simulate(mc_y, T+1; init=y0_ind)
        mc_κ = MarkovChain(Πκ)
        κ_indices[:,s] = simulate(mc_κ, T+1; init=κ0_ind)
    end

    return (y_indices, κ_indices)
end

"""
Here, I simulate the endowment process for all the experiments
"""
# Simulate the edowment shocks that I'll use for all the exercises
(y_indices, κ_indices) = simulate(samano)

In [ ]:
"""
This function realizes S simulations of the ECONOMY for T periods 
"""
function QuantEcon.simulate(ae, T=1000, S=1000, ysim_ind=y_indices, B0=ae.Bgrid[1], A0=ae.Agrid[1], κsim_ind=κ_indices)
        
    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae
    
    # Allocate the values of the simulations in a matrix of dimension (T+1,S)    
    csim_mat = zeros(T+1, S)
    ysim_mat = zeros(T+1, S)
    Asim_mat = zeros(T+1, S)
    Bsim_mat = zeros(T+1, S)
    qsim_mat = zeros(T+1, S)
    default_mat = zeros(Bool, T+1, S)                                                       
    
    for s=1:S
        
        # Get initial indices
        zero_index = searchsortedfirst(Bgrid, 0.)
        B0_ind = searchsortedfirst(Bgrid, B0)
        A0_ind = searchsortedfirst(Agrid, A0)

        # Allocate and Fill output
        # this generate a vector with T+1 values for y folllowing a markov process
        ysim_val = zeros(T+1)

        # Generate vectors for Debt, Reserves, and bonds price
        Bsim_ind = fill(0, T+1)
        Bsim_val = zeros(T+1)
        Asim_ind = fill(0, T+1)
        Asim_val = zeros(T+1)
        csim_val = zeros(T+1)
        qsim_val = zeros(T+1)
        default_status = fill(false, T+1)                                         
        Bsim_ind[1], Asim_ind[1], default_status[1] = B0_ind, A0_ind, false
        ysim_val[1], Bsim_val[1], Asim_val[1] = ygrid[ysim_ind[1,s]], Bgrid[B0_ind], Agrid[A0_ind]

        for t=1:T
            # Get today's indexes
            yi, Bi, Ai, κi = ysim_ind[t,s], Bsim_ind[t], Asim_ind[t], κsim_ind[t,s] 
            defstat = default_status[t]
            
            # If you are not in default
            if !defstat
                default_today = vfr[Bi, yi, Ai, κi] < vfd[1, yi, Ai, κi] 
                    
                if default_today
                    default_status[t] = true
                    default_status[t+1] = true
                    ysim_val[t] = ydefgrid[ysim_ind[t,s]]
                    Asim_ind[t+1] = policyAd[1, yi, Ai, κi]
                    Asim_val[t+1] = Agrid[Asim_ind[t+1]]
                    Bsim_ind[t+1] = zero_index
                    Bsim_val[t+1] = 0.
                    qsim_val[t] = 0.
                    csim_val[t] = ysim_val[t] - g + Asim_val[t] - (exp(-r)*Asim_val[t+1]) 
                else
                    default_status[t] = false
                    ysim_val[t] = ygrid[ysim_ind[t,s]]
                    Asim_ind[t+1] = policyAr[Bi, yi, Ai, κi]
                    Asim_val[t+1] = Agrid[Asim_ind[t+1]]
                    Bsim_ind[t+1] = policyB[Bi, yi, Ai, κi]
                    Bsim_val[t+1] = Bgrid[Bsim_ind[t+1]]
                    qsim_val[t] = q[Bsim_ind[t+1], ysim_ind[t,s], Asim_ind[t+1], κsim_ind[t,s]]
                    csim_val[t] = ysim_val[t] - g + Asim_val[t] - (exp(-r)*Asim_val[t+1]) - Bsim_val[t] + qsim_val[t]*Bsim_val[t+1]   
                end
                
            # If you are in default    
            else
                ysim_val[t] = ydefgrid[ysim_ind[t,s]]
                Asim_ind[t+1] = policyAd[1, yi, Ai, κi]
                Asim_val[t+1] = Agrid[Asim_ind[t+1]]            
                Bsim_ind[t+1] = zero_index
                Bsim_val[t+1] = 0.
                qsim_val[t] = 0.
                csim_val[t] = ysim_val[t] - g + Asim_val[t] - (exp(-r)*Asim_val[t+1]) 
                # With probability θ exit default status
                if rand() < θ
                    default_status[t+1] = false
                else
                    default_status[t+1] = true
                end
            end
        end
        
        # Fill the matrix of dimension (T+1,S) with the vector of dimension T+1 for each s
        ysim_mat[:,s] = ysim_val
        Asim_mat[:,s] = Asim_val
        Bsim_mat[:,s] = Bsim_val
        csim_mat[:,s] = csim_val
        qsim_mat[:,s] = qsim_val
        default_mat[:,s] = default_status
    end
    
    return (csim_mat, ysim_mat, Asim_mat, Bsim_mat, qsim_mat, default_mat) 
    
end

In [ ]:
"""
This function computes key statistics taking the last KT observations of samples in which the last default
was observed at least BT periods before the beginning of the sample. 
"""
function key_statistics!(ae; S=1000, T=1000, KT=50, BT=25)
    
    # unpack stuff
    @unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = ae
    @unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = ae
    
    # Simulate the economy
    (c_sim, y_sim, A_sim, B_sim, q_sim, D_sim) = simulate(ae)
    
    # Identify samples where there's no defaut between period T-KT-25 and T-KT
    dummy_default = zeros(Int64,BT,S)
    for t in 1:BT
        for s in 1:S
            dummy_default[t, s] = D_sim[T-KT-BT+t, s] 
        end
    end
    dummy_samples = zeros(Int64,S)
    for s in 1:S
        if sum(dummy_default[:, s]) > 0
            dummy_samples[s] = 1
        end        
    end
    
    # Keep the last KT periods
    c_erg = zeros(KT,S)
    y_erg = zeros(KT,S)
    D_erg = zeros(Int64,KT,S)
    B_erg = zeros(KT,S)
    A_erg = zeros(KT,S)
    rs_erg = zeros(KT,S)
    
    yind_erg = zeros(Int64,KT,S)
    κind_erg = zeros(Int64,KT,S)                                                                  
    for t=1:KT
        for s=1:S
            D_erg[t, s] = D_sim[T-KT+t, s]
            
            if D_erg[t, s] == 0
                c_erg[t, s] = c_sim[T-KT+t, s]
            else
                c_erg[t, s] = 9999.
            end
            
            if D_erg[t, s] == 0
                y_erg[t, s] = y_sim[T-KT+t, s]
            else
                y_erg[t, s] = 9999.
            end
            
            if D_erg[t, s] == 0
                B_erg[t, s] = B_sim[T-KT+t, s] / y_sim[T-KT+t, s]
            else
                B_erg[t, s] = 9999.
            end
            
            if D_erg[t, s] == 0
                A_erg[t, s] = A_sim[T-KT+t, s] / y_sim[T-KT+t, s]
            else
                A_erg[t, s] = 9999.
            end

            if D_erg[t, s] == 0
                rs_erg[t, s] = 1/q_sim[T-KT+t, s] - exp(r)
            else
                rs_erg[t, s] = 9999.
            end   
            
            yind_erg[t,s] = y_indices[T-KT+t, s]
            κind_erg[t,s] = κ_indices[T-KT+t, s]
        end
    end
    
    # Compute statistics using the last KT periods of dummy samples = 0
    # mean(B/y)
    B_avg = zeros(S)
    # mean(A/y)
    A_avg = zeros(S)
    # cor(A/y, B/y)
    AB_cor = zeros(S)
    # cor(A/y, y)
    Ay_cor = zeros(S)
    # cor(B/y, y)
    By_cor = zeros(S)
    # mean(rs)
    rs_avg = zeros(S)
    # std(rs)
    rs_std = zeros(S)
    # cor(rs,y)
    rsy_cor = zeros(S)
    # cor(rs,A/y)
    rsA_cor = zeros(S)
    # cor(rs,B/y)
    rsB_cor = zeros(S)
    
    # cor(c,y)
    cy_cor = zeros(S)
    # std(c)/std(y)
    cy_std = zeros(S)
    
    default = zeros(S)
    
    for s=1:S
        # Identify simulations where last default was observed at least 25 periods before the begining of the sample
        if dummy_samples[s] == 0
            # Compute statistics for each simulation in repayment states
            c_repayment = filter!(x -> x ≠ 9999, c_erg[:,s])
            y_repayment = filter!(x -> x ≠ 9999, y_erg[:,s])
            B_repayment = filter!(x -> x ≠ 9999, B_erg[:,s])
            A_repayment = filter!(x -> x ≠ 9999, A_erg[:,s])
            rs_repayment = filter!(x -> x ≠ 9999, rs_erg[:,s])
            
            B_avg[s]   = mean(B_repayment) 
            A_avg[s]   = mean(A_repayment)
            AB_cor[s]  = cor(A_repayment,B_repayment)
            Ay_cor[s]  = cor(A_repayment,y_repayment)
            By_cor[s]  = cor(B_repayment,y_repayment)
            rs_avg[s]  = mean(rs_repayment)
            rs_std[s]  = std(rs_repayment)
            rsy_cor[s] = cor(rs_repayment,y_repayment)
            rsA_cor[s] = cor(rs_repayment,A_repayment)
            rsB_cor[s] = cor(rs_repayment,B_repayment)
            
            cy_cor[s]  = cor(c_repayment,y_repayment)
            cy_std[s]  = std(c_repayment)/std(y_repayment)
            
            default[s] = (sum(D_erg[:, s])/50)*100
            
        end
    end
         
    return (c_erg, y_erg, D_erg, B_erg, A_erg, rs_erg, 
            B_avg, A_avg, AB_cor, Ay_cor, By_cor, 
            rs_avg, rs_std, rsy_cor, rsA_cor, rsB_cor,
            cy_cor, cy_std, default, yind_erg, κind_erg)
    
end

In [ ]:
"""
Tables 4-5. 
BENCHMARK CALIBRATION (κ > 0.) PAPER VERSION MAY 2022
INDEPENDENT CENTRAL BANK
βf = 0.939 < βm = .966
"""

(c_sam, y_sam, D_sam, B_sam, A_sam, rs_sam, 
 B_avg_sam, A_avg_sam, AB_cor_sam, Ay_cor_sam, By_cor_sam, 
 rs_avg_sam, rs_std_sam, rsy_cor_sam, rsA_cor_sam, rsB_cor_sam,
 cy_cor_sam, cy_std_sam, default_sam, yind_sam, κind_sam) = key_statistics!(samano)

 Statistics_Samano = [mean(B_avg_sam)*100 mean(rs_avg_sam)*100 mean(rs_std_sam)*100 mean(default_sam);
                      mean(A_avg_sam)*100 mean(AB_cor_sam) mean(Ay_cor_sam) mean(cy_cor_sam);
                      mean(cy_std_sam) mean(rsy_cor_sam) mean(rsB_cor_sam) mean(rsA_cor_sam)]

In [ ]:
"""
Table 6. 
BENCHMARK CALIBRATION (κ > 0.) PAPER VERSION MAY 2022
CONSOLIDATED GOVERNMENT
βf = βm = 0.939
"""

alfaro = Model( βm = .939,     # = βf
                βf = .939,     # Debt to GDP = 44.4% (BY SIMULATION)
                 σ = 2.0,      # Standard parameter value as Alfaro and Kanczuk (2009)
                 r = 0.011,    # U.S. Treasury Bills real interest rate
                d0 = -0.809,   # Mean spread = 267 bp (BY SIMULATION)
                d1 = 0.902,    # Increase in spread = 296 bp (BY SIMULATION)
                 ρ = 0.66,     # Auto-correlation of Mexico's GDP
                 η = 0.034,    # Variance of Mexico's GDP
                 κ = 0.27,     # Probability of default equal to 2% (BY SIMULATION)
                lh = .3,       # Probability of transiting to high risk premium (Global EMBI+)
                hl = .80,      # Probability of transiting to low risk premium (Global EMBI+)
                 θ = 0.11,     # 9 years in default for Mexico (1982-1990)
                 g = 0.,       # Standard parameter value as Alfaro and Kanczuk (2009)
                ny = 11,       
                nB = 31,
                nA = 11)

@time Guess_and_Verify!(alfaro, tol = 1e-8, maxit = 100) 

(c_alf, y_alf, D_alf, B_alf, A_alf, rs_alf, 
 B_avg_alf, A_avg_alf, AB_cor_alf, Ay_cor_alf, By_cor_alf, 
 rs_avg_alf, rs_std_alf, rsy_cor_alf, rsA_cor_alf, rsB_cor_alf,
 cy_cor_alf, cy_std_alf, default_alf, yind_alf, κind_alf) = key_statistics!(alfaro)

 Statistics_Alfaro = [mean(B_avg_alf)*100 mean(rs_avg_alf)*100 mean(rs_std_alf)*100 mean(default_alf);
                      mean(A_avg_alf)*100 mean(AB_cor_alf) mean(Ay_cor_alf) mean(cy_cor_alf);
                      mean(cy_std_alf) mean(rsy_cor_alf) mean(rsB_cor_alf) mean(rsA_cor_alf)] 

In [ ]:
"""
Table 8 Column 3.
ALTERNATIVE CALIBRATION (κ > 0.) PAPER VERSION MAY 2022
INDEPENDENT CENTRAL BANK
βf = 0.954 < βm = .966
"""

alt = Model(βm = 0.966,            # Money msrket interest rate equal to 3.5%
              βf = 0.954,            # Debt to GDP = 44.4% (BY SIMULATION)
               σ = 2.,               # Standard parameter value as Alfaro and Kanczuk (2009)
               r = 0.011,            # U.S. Treasury Bills real interest rat
              d0 = -0.894,           # Mean spread = 273 bp (BY SIMULATION in benchmark calibration)
              d1 = 0.902,            # Increase in spread = 300 bp (BY SIMULATION in benchmark calibration)
               ρ = 0.66,             # Auto-correlation of Mexico's GDP
               η = 0.034,            # Variance of Mexico's GDP
               κ = 0.27,             # Probability of default equal to 3% (BY SIMULATION)
              lh = .3,               # Probability of transiting to high risk premium (Global EMBI+)
              hl = .80,              # Probability of transiting to low risk premium (Global EMBI+)
               θ = 0.11,             # 9 years in default for Mexico (1982-1990)
               g = 0.,               # N/A
              ny = 11,       
              nB = 31,
              nA = 11)

@time Guess_and_Verify!(alt, tol = 1e-8, maxit = 100) 

(c_alt, y_alt, D_alt, B_alt, A_alt, rs_alt, 
 B_avg_alt, A_avg_alt, AB_cor_alt, Ay_cor_alt, By_cor_alt, 
 rs_avg_alt, rs_std_alt, rsy_cor_alt, rsA_cor_alt, rsB_cor_alt,
 cy_cor_alt, cy_std_alt, default_alt, yind_alt, κind_alt) = key_statistics!(alt)

 Statistics_alt=[mean(A_avg_alt)*100 mean(B_avg_alt)*100 mean(AB_cor_alt) mean(Ay_cor_alt) mean(By_cor_alt); 
                mean(rs_avg_alt)*100 mean(rs_std_alt)*100 mean(rsy_cor_alt) mean(rsA_cor_alt) mean(rsB_cor_alt);
                mean(cy_cor_alt) mean(cy_std_alt) mean(default_alt) 0 0] 

In [ ]:
"""
Table 8 Column 4.
ALTERNATIVE CALIBRATION (κ = 0.) PAPER VERSION MAY 2022
INDEPENDENT CENTRAL BANK
βf = 0.954 < βm = .966
"""

altκ0 = Model(βm = 0.966,            # Money msrket interest rate equal to 3.5%
              βf = 0.954,            # Debt to GDP = 44.4% (BY SIMULATION)
               σ = 2.,               # Standard parameter value as Alfaro and Kanczuk (2009)
               r = 0.011,            # U.S. Treasury Bills real interest rat
              d0 = -0.894,           # Mean spread = 273 bp (BY SIMULATION in benchmark calibration)
              d1 = 0.902,            # Increase in spread = 300 bp (BY SIMULATION in benchmark calibration)
               ρ = 0.66,             # Auto-correlation of Mexico's GDP
               η = 0.034,            # Variance of Mexico's GDP
               κ = 0.,               # n/a
              lh = .3,               # Probability of transiting to high risk premium (Global EMBI+)
              hl = .80,              # Probability of transiting to low risk premium (Global EMBI+)
               θ = 0.11,             # 9 years in default for Mexico (1982-1990)
               g = 0.,               # N/A
              ny = 11,       
              nB = 31,
              nA = 11)

@time Guess_and_Verify!(altκ0, tol = 1e-8, maxit = 100) 

(c_altκ0, y_altκ0, D_altκ0, B_altκ0, A_altκ0, rs_altκ0, 
 B_avg_altκ0, A_avg_altκ0, AB_cor_altκ0, Ay_cor_altκ0, By_cor_altκ0, 
 rs_avg_altκ0, rs_std_altκ0, rsy_cor_altκ0, rsA_cor_altκ0, rsB_cor_altκ0,
 cy_cor_altκ0, cy_std_altκ0, default_altκ0, yind_altκ0, κind_altκ0) = key_statistics!(altκ0)

 Statistics_altκ0=[mean(A_avg_altκ0)*100 mean(B_avg_altκ0)*100 mean(AB_cor_altκ0) mean(Ay_cor_altκ0) mean(By_cor_altκ0); 
                mean(rs_avg_altκ0)*100 mean(rs_std_altκ0)*100 mean(rsy_cor_altκ0) mean(rsA_cor_altκ0) mean(rsB_cor_altκ0);
                mean(cy_cor_altκ0) mean(cy_std_altκ0) mean(default_altκ0) 0 0] 

In [ ]:
"""
PLOTS
"""

using DataFrames, Plots
gr(fmt=:png)

In [ ]:
"""
To plot Figure 3, we need to simulate an economy populated by a benevolent social planner
Figure 3.
BENCHMARK CALIBRATION (κ > 0.) PAPER VERSION MAY 2022
BENEVOLENT SOCIAL PLANNER
βf = βm = .966
"""

optimal = Model(βm = 0.966,            # Money msrket interest rate equal to 3.5%
                βf = 0.966,            # = βm
                 σ = 2.,               # Standard parameter value as Alfaro and Kanczuk (2009)
                 r = 0.011,            # U.S. Treasury Bills real interest rat
                d0 = -0.894,           # Mean spread = 273 bp (BY SIMULATION in benchmark calibration)
                d1 = 0.902,            # Increase in spread = 300 bp (BY SIMULATION in benchmark calibration)
                 ρ = 0.66,             # Auto-correlation of Mexico's GDP
                 η = 0.034,            # Variance of Mexico's GDP
                 κ = 0.27,             # n/a
                lh = .3,               # Probability of transiting to high risk premium (Global EMBI+)
                hl = .80,              # Probability of transiting to low risk premium (Global EMBI+)
                 θ = 0.11,             # 9 years in default for Mexico (1982-1990)
                 g = 0.,               # N/A
                ny = 11,       
                nB = 31,
                nA = 11)

@time Guess_and_Verify!(optimal, tol = 1e-8, maxit = 100) 

(c_opt, y_opt, D_opt, B_opt, A_opt, rs_opt, 
 B_avg_opt, A_avg_opt, AB_cor_opt, Ay_cor_opt, By_cor_opt, 
 rs_avg_opt, rs_std_opt, rsy_cor_opt, rsA_cor_opt, rsB_cor_opt,
 cy_cor_opt, cy_std_opt, default_opt, yind_opt, κind_opt) = key_statistics!(optimal)

 Statistics_opt=[mean(A_avg_opt)*100 mean(B_avg_opt)*100 mean(AB_cor_opt) mean(Ay_cor_opt) mean(By_cor_opt); 
                mean(rs_avg_opt)*100 mean(rs_std_opt)*100 mean(rsy_cor_opt) mean(rsA_cor_opt) mean(rsB_cor_opt);
                mean(cy_cor_opt) mean(cy_std_opt) mean(default_opt) 0 0] 

In [ ]:
"""
First plot the alfaro economy
"""

# unpack stuff
@unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = alfaro
@unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = alfaro

# extract a suAtable plot grid
x = zeros(0)
c_alf_lowk = zeros(0)
c_alf_high = zeros(0)

for i in 1:ny
    a = ygrid[i]
    push!(x, a)
    push!(c_alf_lowk, ygrid[i] + Agrid[1] - Bgrid[8] + q[policyB[8, i, 1, 1],i,policyAr[8, i, 1, 1],1]*Bgrid[policyB[8, i, 1, 1]] - exp(-r)*Agrid[policyAr[8, i, 1, 1]])
    push!(c_alf_high, ygrid[i] + Agrid[1] - Bgrid[8] + q[policyB[8, i, 1, 2],i,policyAr[8, i, 1, 2],2]*Bgrid[policyB[8, i, 1, 2]] - exp(-r)*Agrid[policyAr[8, i, 1, 2]])

end


# generate plot
plot(x, c_alf_lowk, label = "Consolidated Government", legendfontsize=:12, ls=:dot, lw=3, c=:red)
#plot!(x, c_ben_lowk, label = "Benevolent Social Planner", legendfontsize=:12, ls=:solid, lw=3, c=:green)
#plot!(x, x, label = "45'", legendfontsize=:12, ls=:dash, lw=1, c=:black)
plot!(xlabel = "y", ylabel = "c(y)", xlims = (.95,1.052), ylims = (.90,1.152), guidefontsize=:12,
      tickfontsize=:12, xticks = .95:0.01:1.055, yticks = .90:0.05:1.152, fg_color_grid=:white, legend = :topright)

In [ ]:
"""
Figure 3a
"""

# unpack stuff
@unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = optimal
@unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = optimal

# extract a suAtable plot grid
x = zeros(0)
c_ben_lowk = zeros(0)
c_ben_high = zeros(0)

for i in 1:ny
    a = ygrid[i]
    push!(x, a)
    push!(c_ben_lowk, ygrid[i] + Agrid[1] - Bgrid[8] + q[policyB[8, i, 1, 1],i,policyAr[8, i, 1, 1],1]*Bgrid[policyB[8, i, 1, 1]] - exp(-r)*Agrid[policyAr[8, i, 1, 1]])
    push!(c_ben_high, ygrid[i] + Agrid[1] - Bgrid[8] + q[policyB[8, i, 1, 2],i,policyAr[8, i, 1, 2],2]*Bgrid[policyB[8, i, 1, 2]] - exp(-r)*Agrid[policyAr[8, i, 1, 2]])

end


# generate plot
plot(x, c_alf_lowk, label = "Consolidated Government", legendfontsize=:12, ls=:dot, lw=3, c=:red)
plot!(x, c_ben_lowk, label = "Benevolent Social Planner", legendfontsize=:12, ls=:solid, lw=3, c=:green)
#plot!(x, x, label = "45'", legendfontsize=:12, ls=:dash, lw=1, c=:black)
plot!(xlabel = "y", ylabel = "c(y)", xlims = (.95,1.052), ylims = (.90,1.152), guidefontsize=:12,
      tickfontsize=:12, xticks = .95:0.01:1.055, yticks = .90:0.05:1.152, fg_color_grid=:white, legend = :topright)

In [ ]:
"""
Figure 3b
"""

# extract a suAtable plot grid
x = zeros(0)
N_alf_lowk = zeros(0)
N_ben_lowk = zeros(0)
N_tge_lowk = zeros(0)
for i in 1:alfaro.ny
    a = alfaro.ygrid[i]
    push!(x, a)
    push!(N_alf_lowk, alfaro.Bgrid[alfaro.policyB[8, i, 1, 1]] - alfaro.Agrid[alfaro.policyAr[8, i, 1, 1]]-alfaro.Bgrid[8] + alfaro.Agrid[1])
    push!(N_ben_lowk, optimal.Bgrid[optimal.policyB[8, i, 1, 1]] - optimal.Agrid[optimal.policyAr[8, i, 1, 1]]-optimal.Bgrid[8] + optimal.Agrid[1])
    #push!(N_tge_lowk, samano.Bgrid[samano.policyB[23, i, 5, 1]] - samano.Agrid[samano.policyAr[23, i, 5, 1]]-samano.Bgrid[23] + samano.Agrid[5])
end

# define overborrowing as the difference between D' and D*
#B_over = B_ind - B_opt

# generate plot
plot(x, N_alf_lowk*100, label = "Consolidated Government", legendfontsize=:12, ls=:dot, lw=3, c=:red)
#plot!(x, N_tge_lowk, label = "Two Government Entities", legendfontsize=:12, ls=:solid, lw=5, c=:green)
plot!(x, N_ben_lowk*100, label = "Benevolent Social Planner", legendfontsize=:12, ls=:solid, lw=3, c=:green)
plot!(xlabel = "y", ylabel = "Change in Net Debt (% y)", xlims = (.95,1.055), ylims = (-10,15), guidefontsize=:12,
      tickfontsize=:12, xticks = .95:0.01:1.05, yticks = -10:5:15, fg_color_grid=:white, legend = :topright)

In [ ]:
"""
Figure 4a
"""

# extract a suitable plot grid
x = zeros(0)
q_reserves = zeros(0)
q_publdebt = zeros(0)
for i in 1:samano.nA
    a = samano.Agrid[i]
    push!(x, a)
    push!(q_reserves, samano.q[23, 4, i, 1])
    push!(q_publdebt, samano.q[i, 4, 5, 1])
end

# generate plot
plot(x, q_publdebt, label = "q(B'; A'=avg, y=avg, k=0)", legendfontsize=:12, ls=:dot, lw=3, c=:red)
plot!(x, q_reserves, label = "q(A'; B'=avg, y=avg, k=0)", legendfontsize=:12, ls=:solid, lw=3, c=:green)
plot!(xlabel = "Debt (B'), Reserves (A')", ylabel = "Bond Price Schedule, q", xlims = (0,.61), ylims = (0.5,1), guidefontsize=:12,
      tickfontsize=:12, xticks = 0:0.1:.6, yticks = 0.5:0.1:1, fg_color_grid=:white, legend = :bottomleft)

In [ ]:
"""
Figure 4b
"""

# extract a suitable plot grid
x = zeros(0)
vfd = zeros(0)
vfr = zeros(0)
for i in 1:samano.nA
    a = samano.Agrid[i]
    push!(x, a)
    push!(vfd, samano.vfd[1, 4, i, 1])
    push!(vfr, samano.vfr[23, 4, i, 1])
end

# generate plot
plot(x, vfd, label = "Default Value", legendfontsize=:12, ls=:dot, lw=3, c=:red)
plot!(x, vfr, label = "Repayment Value", legendfontsize=:12, ls=:solid, lw=3, c=:green)
plot!(xlabel = "A", ylabel = "Fiscal Authority's Value Function", xlims = (0,.6), ylims = (-19.4,-18.4), guidefontsize=:12,
      tickfontsize=:12, xticks = 0:0.1:.6, yticks = -19.4:.2:-18.4, fg_color_grid=:white, legend = :bottomleft)

In [ ]:
"""
Figure 5a
"""

# unpack stuff
@unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = samano
@unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = samano

L = 5;
H = 6;

# equilibrium level of consumption
c_low = ygrid[L] + Agrid[5] - Bgrid[23] + q[policyB[23,L,5,1],L,policyAr[23,L,5,1],1]*Bgrid[policyB[23,L,5,1]] - exp(-r)*Agrid[policyAr[23,L,5,1]]
c_high = ygrid[H] + Agrid[5] - Bgrid[23] + q[policyB[23,H,5,1],H,policyAr[23,H,5,1],1]*Bgrid[policyB[23,H,5,1]] - exp(-r)*Agrid[policyAr[23,H,5,1]]

#q[searchsortedfirst(Bgrid, (c_mean+exp(-r)*Agrid[i]-Agrid[5]+Bgrid[23]-ygrid[6])/q[policyB[23,6,5,1],6,policyAr[23,6,5,1],1]),6,i,1]

x = zeros(0)
B_low = zeros(0)
A_low = zeros(0)
B_low_eq = zeros(0)
A_low_eq = zeros(0)
B_high = zeros(0)
A_high = zeros(0)
B_high_eq = zeros(0)
A_high_eq = zeros(0)
for i in 1:11
    a = Agrid[i]
    push!(A_low, a)
    push!(B_low, (c_low - ygrid[L] - Agrid[5] + Bgrid[23] + exp(-r)*Agrid[i])/q[policyB[23,L,5,1],L,i,1])
    push!(A_low_eq, Agrid[policyAr[23,L,5,1]])
    push!(B_low_eq, Bgrid[policyB[23,L,5,1]])
    push!(A_high, a)
    push!(B_high, (c_high - ygrid[H] - Agrid[5] + Bgrid[23] + exp(-r)*Agrid[i])/q[policyB[23,H,5,1],H,i,1])
    push!(B_high_eq, Bgrid[policyB[23,H,5,1]])
    push!(A_high_eq, Agrid[policyAr[23,H,5,1]])
    
end

# generate plot
plot(A_low, B_low, label = "(B',A') for low y", legendfontsize=:12, ls=:dot, lw=3, c=:red)
scatter!(A_low_eq, B_low_eq, label = "(B',A') in equilibrium", legendfontsize=:12, ls=:dot, lw=5, c=:red)
plot!(A_high, B_high, label = "(B',A') for high y", legendfontsize=:12, ls=:solid, lw=3, c=:green)
scatter!(A_high_eq, B_high_eq, label = "(B',A') in equilibrium", legendfontsize=:12, ls=:dot, lw=5, c=:green)
plot!(xlabel = "Reserves (A')", ylabel = "Debt (B')", xlims = (-0.01,.21), ylims = (0.3,.61), guidefontsize=:12,
      tickfontsize=:12, xticks = 0.0:0.04:.21, yticks = 0.3:0.05:.61, fg_color_grid=:white, legend = :topleft)

In [ ]:
"""
Figure 5b
"""

# unpack stuff
@unpack βm, βf, σ, r, d0, d1, ρ, η, κ, lh, hl, θ, g, ny, nB, nA = samano
@unpack ygrid, ydefgrid, Bgrid, Agrid, Πy, Πκ, vmd, vmr, vf, vfd, vfr, 
            policyAd, policyAr, policyB, policyD, q, defprob = samano

L = 5;
H = 6;

# equilibrium level of consumption
c_low = ygrid[L] + Agrid[5] - Bgrid[23] + q[policyB[23,L,5,1],L,policyAr[23,L,5,1],1]*Bgrid[policyB[23,L,5,1]] - exp(-r)*Agrid[policyAr[23,L,5,1]]
c_high = ygrid[H] + Agrid[5] - Bgrid[23] + q[policyB[23,H,5,1],H,policyAr[23,H,5,1],1]*Bgrid[policyB[23,H,5,1]] - exp(-r)*Agrid[policyAr[23,H,5,1]]

#q[searchsortedfirst(Bgrid, (c_mean+exp(-r)*Agrid[i]-Agrid[5]+Bgrid[23]-ygrid[6])/q[policyB[23,6,5,1],6,policyAr[23,6,5,1],1]),6,i,1]

x = zeros(0)
q_low = zeros(0)
A_low = zeros(0)
q_low_eq = zeros(0)
A_low_eq = zeros(0)
q_high = zeros(0)
A_high = zeros(0)
q_high_eq = zeros(0)
A_high_eq = zeros(0)
for i in 1:11
    a = Agrid[i]
    push!(A_low, a)
    push!(q_low, (1/(q[searchsortedfirst(Bgrid,B_low[i]),L,i,1]))-exp(r))
    push!(A_low_eq, Agrid[policyAr[23,L,5,1]])
    push!(q_low_eq, (1/q[policyB[23,L,5,1], L, policyAr[23,L,5,1], 1])-exp(r))
    push!(A_high, a)
    push!(q_high, (1/(q[searchsortedfirst(Bgrid,B_high[i]),H,i,1]))-exp(r))
    push!(A_high_eq, Agrid[policyAr[23,H,5,1]])
    push!(q_high_eq, (1/q[searchsortedfirst(Bgrid,B_low[7]), H, 7, 1])-exp(r))    
    #push!(q_high_eq, (1/q[policyB[23,H,5,1], H, policyAr[23,H,5,1], 1])-exp(r))
end

# generate plot
plot(A_low, q_low, label = "(B',A') for low y", legendfontsize=:12, ls=:dot, lw=3, c=:red)
scatter!(A_low_eq, q_low_eq, label = "(B',A') in equilibrium", legendfontsize=:12, ls=:dot, lw=5, c=:red)
plot!(A_high, q_high, label = "(B',A') for high y", legendfontsize=:12, ls=:solid, lw=3, c=:green)
scatter!(A_high_eq, q_high_eq, label = "(B',A') in equilibrium", legendfontsize=:12, ls=:dot, lw=5, c=:green)
plot!(xlabel = "Reserves (A')", ylabel = "Spread (%)", xlims = (-0.01,.21), ylims = (-0.01,.31), guidefontsize=:12,
      tickfontsize=:12, xticks = 0.0:0.04:.21, yticks = 0:0.05:.3, fg_color_grid=:white, legend = :topleft)